In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import xml.etree.ElementTree as ET

# To convert the annotation files form xml to txt to train YOLO Model
def convert_annotation(xml_file, txt_output, img_width, img_height):
    # Parse the XML files and handle the errors
    try:
        tree = ET.parse(xml_file)
        root = tree.getroot()
    except ET.ParseError as e:
        print(f"Error parsing XML file {xml_file}: {e}")
        return

    with open(txt_output, "w") as f:
        for obj in root.findall("object"):
            label = obj.find("name").text
            xmin = int(obj.find("bndbox/xmin").text)
            ymin = int(obj.find("bndbox/ymin").text)
            xmax = int(obj.find("bndbox/xmax").text)
            ymax = int(obj.find("bndbox/ymax").text)

            # YOLO format calculation
            x_center = (xmin + xmax) / 2 / img_width
            y_center = (ymin + ymax) / 2 / img_height
            box_width = (xmax - xmin) / img_width
            box_height = (ymax - ymin) / img_height

            f.write(f"0 {x_center} {y_center} {box_width} {box_height}\n")

In [ ]:
xml_folder = '/content/drive/MyDrive/Data For Project/Data/annotations'
txt_output_folder = '/content/drive/MyDrive/Data For Project/Data/YoloVersion'
image_folder = '/content/drive/MyDrive/Data For Project/Data/images'

import cv2
import os

for xml_filename in os.listdir(xml_folder):
    xml_file_path = os.path.join(xml_folder, xml_filename)
    base_name = os.path.splitext(xml_filename)[0]

    # find image file
    image_path = os.path.join(image_folder, f"{base_name}.png")
    image = cv2.imread(image_path)
    img_height, img_width, _ = image.shape

    txt_output_path = os.path.join(txt_output_folder, f"{base_name}.txt")

    convert_annotation(xml_file_path, txt_output_path, img_width, img_height)


In [3]:
import os
import shutil

def split_dataset(image_dir, annotation_dir, train_dir, test_dir):
    images = [f for f in os.listdir(image_dir)]

    split_index = int(len(images) * 0.9)

    # Copy images and annotations to training set and test set
    for image in images[:split_index]:
        shutil.copy(os.path.join(image_dir, image), os.path.join(train_dir, image))
        annotation_file = image.replace('.png', '.txt')
        shutil.copy(os.path.join(annotation_dir, annotation_file), os.path.join(train_dir, annotation_file))

    for image in images[split_index:]:
        shutil.copy(os.path.join(image_dir, image), os.path.join(test_dir, image))
        annotation_file = image.replace('.jpg', '.txt').replace('.png', '.txt')
        shutil.copy(os.path.join(annotation_dir, annotation_file), os.path.join(test_dir, annotation_file))

    print(f"Split completed: {split_index} training images and {len(images) - split_index} testing images.")



In [ ]:
image_directory = '/content/drive/MyDrive/Data For Project/Data/images'
annotation_directory = '/content/drive/MyDrive/Data For Project/Data/YoloVersion'
train_directory = '/content/drive/MyDrive/Data For Project/Data/train'
test_directory = '/content/drive/MyDrive/Data For Project/Data/test'

split_dataset(image_directory, annotation_directory, train_directory, test_directory)

Split completed: 596 training images and 67 testing images.


In [4]:
data_yaml_content = """
train: /content/drive/MyDrive/Data For Project/Data/train
val: /content/drive/MyDrive/Data For Project/Data/test

nc: 1
names: ['pothole']
"""

with open('/content/drive/MyDrive/Data For Project/Data/data.yaml', 'w') as f:
    f.write(data_yaml_content)

In [5]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!git clone https://github.com/ultralytics/yolov5  # clone
!cd yolov5

Looking in indexes: https://download.pytorch.org/whl/cu121
Cloning into 'yolov5'...
remote: Enumerating objects: 17045, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 17045 (delta 10), reused 18 (delta 8), pack-reused 17022 (from 1)
Receiving objects: 100% (17045/17045), 15.67 MiB | 16.38 MiB/s, done.
Resolving deltas: 100% (11704/11704), done.


In [6]:
import yaml
import torch

model_config_path = '/content/yolov5/models/yolov5s.yaml'

with open(model_config_path) as f:
    model_config = yaml.safe_load(f)

model_config['nc'] += 1

with open(model_config_path, 'w') as f:
    yaml.dump(model_config, f)

In [7]:
%cd /content/yolov5/
!python train.py --img 640 --batch 16 --epochs 100 --data "/content/drive/MyDrive/Data For Project/Data/data.yaml" --cfg ./models/yolov5m.yaml --weights yolov5m.pt --workers 2

Streaming output truncated to the last 5000 lines.
  with torch.cuda.amp.autocast(amp):
      38/99      6.98G    0.02745    0.01858          0         91        640:  42% 16/38 [00:16<00:27,  1.23s/it]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      38/99      6.98G     0.0274    0.01868          0         85        640:  45% 17/38 [00:19<00:37,  1.77s/it]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      38/99      6.98G    0.02764    0.01909          0        100        640:  47% 18/38 [00:19<00:27,  1.36s/it]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):


In [8]:
!python val.py --weights /content/yolov5/runs/train/exp/weights/best.pt --data "/content/drive/MyDrive/Data For Project/Data/data.yaml" --img 640 --batch 16 --task val


val: data=/content/drive/MyDrive/Data For Project/Data/data.yaml, weights=['/content/yolov5/runs/train/exp/weights/best.pt'], batch_size=16, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-383-g1435a8ee Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5m summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
val: Scanning /content/drive/MyDrive/Data For Project/Data/test.cache... 67 images, 0 backgrounds, 0 corrupt: 100% 67/67 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95:   0% 0/5 [00:00<?, ?it/s]WARNING ⚠️ NMS time limit 1.300s exceeded
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 5/5 [00:06<00:00

In [9]:
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --source "/content/drive/MyDrive/Data For Project/testVideos/1041636346-preview.mp4" --conf-thres 0.6

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/Data For Project/testVideos/1041636346-preview.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.6, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-383-g1435a8ee Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5m summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded
video 1/1 (1/244) /content/drive/MyDrive/Data For Project/testVideos/1041636346-preview.mp4: 352x640 2 potholes, 54.9ms
video 1/1 (2/244) /content/drive/MyDrive/Data For Project/testVideos/1041636346-

In [10]:
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --source "/content/drive/MyDrive/Data For Project/testVideos/1078652798-preview.mp4" --conf-thres 0.6

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/Data For Project/testVideos/1078652798-preview.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.6, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-383-g1435a8ee Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5m summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded
video 1/1 (1/526) /content/drive/MyDrive/Data For Project/testVideos/1078652798-preview.mp4: 384x640 2 potholes, 55.2ms
video 1/1 (2/526) /content/drive/MyDrive/Data For Project/testVideos/1078652798-

In [11]:
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --source "/content/drive/MyDrive/Data For Project/testVideos/istockphoto-1395835341-640_adpp_is.mp4" --conf-thres 0.6

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/Data For Project/testVideos/istockphoto-1395835341-640_adpp_is.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.6, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-383-g1435a8ee Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5m summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
video 1/1 (1/369) /content/drive/MyDrive/Data For Project/testVideos/istockphoto-1395835341-640_adpp_is.mp4: 384x640 3 potholes, 53.8ms
video 1/1 (2/369) /content/drive/MyDrive/Data For Project/testVideos/istockphoto-139583534